In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from typing import Tuple, List
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.agents import AgentExecutor
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

In [3]:
# import os

# os.environ["OPENAI_API_KEY"] = "your key" # OpenAI API KEY (https://platform.openai.com/api-keys)
# os.environ["TAVILY_API_KEY"] = "your key" # Tavily Search API KEY (https://app.tavily.com/home)

from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily Search API : 효율적이고 빠른 검색 결과를 목표로 하는 LLM 및 RAG에 최적화된 검색 엔진
search = TavilySearchResults(k=3)
search.invoke("0koang velog 주소")

[{'url': 'https://velog.io/@kwon0koang/about',
  'content': 'kwon0koang님의 자기소개가 비었습니다. 서비스 핵심 가치를 이해하고, 지속적인 개선을 이끄는 엔지니어(를 지향함)'},
 {'url': 'https://velog.io/@kwon0koang/posts',
  'content': '스토리텔링과 시각화를 결합하여 팀이 고객 요구를 이해하고 대응할 수 있도록 돕는 도구. 사용자의 목표와 행동을 타임라인 구조로 펼침으로써 시작하며, 생각과 감정을 추가하여 이야기를 만들고 이를 시각화. Customer Journey Map Growth Hacking 고객여정지도 그로스 ...'},
 {'url': 'https://www.youtube.com/watch?v=NTs07_hi0j4',
  'content': '[Hong Kong vlog] a day at Disneyland 💫 | Rides, Show, and Fireworks 🎆 | Hello everyone! Welcome to my channel 😊Thankyou so much for being here. I hope you...'},
 {'url': 'https://alpha.velog.io/@kwon0koang',
  'content': 'kwon0koang님이 작성한 포스트 시리즈들을 확인해보세요. 서비스 핵심 가치를 이해하고, 지속적인 개선을 이끄는 엔지니어(를 지향함)'},
 {'url': 'https://www.youtube.com/watch?v=tBeaUb1hZLc',
  'content': "Our friends were over from New Zealand, so we went on an excursion to Tai O, Hong Kong's authentic fishing village with rows of stilted homes. We ended the a..."}]

In [5]:
from langchain_openai import ChatOpenAI
from langchain_google_vertexai import ChatVertexAI

# llm = ChatOpenAI(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# chat = ChatVertexAI(model="gemini-pro", convert_system_message_to_human=True)

In [6]:
tools = [search]

llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"content : {response.content} / tool_calls : {response.tool_calls}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}